<a href="https://colab.research.google.com/github/henry-bokyum-kim/NNStudy/blob/master/RL_NN_Impl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!apt-get install x11-utils

import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40)

import math
import glob
import io
import base64
from IPython.display import HTML

from pyvirtualdisplay import Display
from IPython import display as ipythondisplay

display = Display(visible=0, size=(1400,900),)
display.start()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libxxf86dga1
Suggested packages:
  mesa-utils
The following NEW packages will be installed:
  libxxf86dga1 x11-utils
0 upgraded, 2 newly installed, 0 to remove and 7 not upgraded.
Need to get 209 kB of archives.
After this operation, 711 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libxxf86dga1 amd64 2:1.1.4-1 [13.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 x11-utils amd64 7.7+3build1 [196 kB]
Fetched 209 kB in 2s (126 kB/s)
Selecting previously unselected package libxxf86dga1:amd64.
(Reading database ... 148036 files and directories currently installed.)
Preparing to unpack .../libxxf86dga1_2%3a1.1.4-1_amd64.deb ...
Unpacking libxxf

<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

HIDDEN = 128
BATCH_SIZE = 32
PERCENTILE = 50

class Net(nn.Module):
  def __init__(self, obs_size, hidden, action_size):
    super(Net, self).__init__()
    self.net=nn.Sequential(
        nn.Linear(obs_size, hidden),
        nn.ReLU(),
        nn.Linear(hidden, action_size),
        nn.Softmax(dim=1)
    )

  def forward(self, x):
    return self.net(x)

In [0]:
def get_batch(batch_size, env):
  batch = []
  reward = 0.0
  obs = env.reset()
  steps = []
  
  while True:
    obs_v = torch.FloatTensor([obs])
    action_v = net(obs_v).data.numpy()[0]
    action = np.random.choice(env.action_space.n, p=action_v)

    next_obs, rew, done, _ = env.step(action)
    reward += rew
    steps.append((obs,action))

    if done:
      batch.append((reward, steps))
      reward = 0.0
      next_obs = env.reset()
      steps = []
      if len(batch) == batch_size:
        yield batch
        batch=[]
      obs = next_obs

In [0]:
def filter_batch(batch):
  rews = list(map(lambda episode: episode[0],batch))
  rews_threshold = np.percentile(rews, PERCENTILE)
  rews_mean = np.mean(rews)

  ret_action = []
  ret_obs = []

  for scene in batch:
    if scene[0] < rews_threshold:
      continue
    ret_obs.extend(map(lambda step: step[0], scene[1]))
    ret_action.extend(map(lambda step: step[1], scene[1]))

  return torch.FloatTensor(ret_obs), torch.FloatTensor(ret_action), rews_threshold, rews_mean

In [0]:
env = gym.make("CartPole-v0")
obs = env.reset()
net = Net(env.observation_space.shape[0], HIDDEN, env.action_space.n)



In [69]:
for i,batch in enumerate(get_batch(BATCH_SIZE, env)):
  obs, actions, threshold, mean = filter_batch(batch)
  if i == 0:
    print(obs.shape, actions.shape, threshold, mean)
    break

torch.Size([496, 4]) torch.Size([496]) 19.5 21.9375
